In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add src to path
sys.path.append('../src')

from data_loader import MRIDataLoader
from pca_analysis import PCAAnalyzer
from segmentation import TissueSegmenter, compare_segmentations
from visualization import *
from utils import create_synthetic_mri_data, compute_tissue_statistics
from main import SegmentationPipeline

# Set random seed for reproducibility
np.random.seed(42)

%matplotlib inline

## 1. Load and Visualize Data

Load MRI data from the `data/raw` folder. If no real data is available, we'll use synthetic data for demonstration.

In [ ]:
# Option 1: Load real MRI data (uncomment when you have data)
# data_loader = MRIDataLoader('../data/raw')
# volume = data_loader.load_nifti('../data/raw/your_mri_file.nii.gz')
# image = data_loader.extract_2d_slice(volume, slice_idx=volume.shape[2]//2, axis=2)
# image = data_loader.normalize_intensity(image)

# Option 2: Create synthetic data for demonstration
print("Creating synthetic MRI data...")
image, ground_truth = create_synthetic_mri_data(shape=(256, 256), n_tissues=3, noise_level=0.1)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.imshow(image, cmap='gray')
ax1.set_title('MRI Image')
ax1.axis('off')

ax2.imshow(ground_truth, cmap='viridis')
ax2.set_title('Ground Truth (if available)')
ax2.axis('off')
plt.tight_layout()
plt.show()

print(f"Image shape: {image.shape}")
print(f"Intensity range: [{image.min():.3f}, {image.max():.3f}]")

## 2. Mathematical Formulation: Eigenanalysis

### Theory

Given MRI image pixels represented as vectors $\mathbf{x}_i \in \mathbb{R}^p$ for $i = 1, \ldots, n$:

1. **Stack samples**: $\mathbf{X} \in \mathbb{R}^{n \times p}$
2. **Center data**: $\mathbf{X}_{centered} = \mathbf{X} - \bar{\mathbf{X}}$
3. **Covariance matrix**: $\mathbf{C} = \frac{1}{n-1} \mathbf{X}^T \mathbf{X}$
4. **Eigendecomposition**: $\mathbf{C}\mathbf{v}_j = \lambda_j \mathbf{v}_j$
5. **Project data**: $\mathbf{Y} = \mathbf{X} \mathbf{V}_k$ where $\mathbf{V}_k$ contains $k$ eigenvectors with largest eigenvalues
6. **Cluster in PC space** to segment tissue types

## 3. Run Complete Segmentation Pipeline

In [ ]:
# Initialize pipeline
n_components = 5  # Number of principal components
n_clusters = 3    # Number of tissue classes (e.g., CSF, gray matter, white matter)

pipeline = SegmentationPipeline(n_components=n_components, n_clusters=n_clusters)

# Run pipeline
print("Running segmentation pipeline...")
results = pipeline.run_pipeline(image)

print("\n✓ Pipeline complete!")

## 4. Analyze Eigenvalues and Explained Variance

In [ ]:
# Plot explained variance
plot_explained_variance(results['pca'], save_path='../results/figures/explained_variance.png')

# Print variance statistics
print("\nVariance Explained by Components:")
for i, (var_ratio, cum_var) in enumerate(zip(
    results['pca'].explained_variance_ratio_,
    results['pca'].get_cumulative_variance()
)):
    print(f"PC {i+1}: {var_ratio:.4f} (cumulative: {cum_var:.4f})")

## 5. Visualize Segmentation Results

In [ ]:
# Plot segmentation
plot_segmentation(
    results['image'],
    results['segmentation'],
    title='Tissue Segmentation (PCA Space)',
    save_path='../results/figures/segmentation_pca.png'
)

## 6. Visualize Data in PC Space

In [ ]:
# Sample points for visualization (full dataset may be too large)
n_samples = min(10000, len(results['Y_pca']))
idx_sample = np.random.choice(len(results['Y_pca']), n_samples, replace=False)

plot_pca_projection_2d(
    results['Y_pca'][idx_sample],
    results['labels'][idx_sample],
    save_path='../results/figures/pca_projection.png'
)

## 7. Compare with Original Space Segmentation

In [ ]:
# Segment in original space for comparison
from segmentation import segment_with_original_space

print("Performing segmentation in original space...")
labels_original = segment_with_original_space(
    pipeline.X_original,
    n_clusters=n_clusters,
    method='gmm'
)
seg_original = labels_original.reshape(image.shape)

# Compare
plot_comparison(
    image,
    results['segmentation'],
    seg_original,
    save_path='../results/figures/comparison.png'
)

# Quantitative comparison
comparison_metrics = compare_segmentations(results['labels'], labels_original)
print("\nComparison Metrics:")
for metric, value in comparison_metrics.items():
    print(f"{metric}: {value:.4f}")

## 8. Tissue Statistics

In [ ]:
# Display tissue statistics
import pandas as pd

stats_df = pd.DataFrame(results['statistics']).T
print("\nTissue Statistics:")
print(stats_df.to_string())

# Visualize tissue distribution
fig, ax = plt.subplots(figsize=(10, 6))
tissues = list(results['statistics'].keys())
percentages = [results['statistics'][t]['percentage'] for t in tissues]

ax.bar(tissues, percentages)
ax.set_ylabel('Percentage (%)')
ax.set_title('Tissue Distribution')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/tissue_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Explore Different Numbers of Components

In [ ]:
# Test different numbers of components
component_counts = [2, 3, 5, 7, 10]
results_by_n_components = {}

for n_comp in component_counts:
    print(f"\nTesting with {n_comp} components...")
    pipe = SegmentationPipeline(n_components=n_comp, n_clusters=n_clusters)
    res = pipe.run_pipeline(image)
    results_by_n_components[n_comp] = res

# Visualize results
fig, axes = plt.subplots(2, len(component_counts), figsize=(20, 8))

for i, n_comp in enumerate(component_counts):
    res = results_by_n_components[n_comp]
    
    axes[0, i].imshow(image, cmap='gray')
    axes[0, i].set_title(f'Original')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(res['segmentation'], cmap='viridis')
    axes[1, i].set_title(f'{n_comp} PCs')
    axes[1, i].axis('off')

plt.suptitle('Segmentation Results with Different Numbers of Components', fontsize=16)
plt.tight_layout()
plt.savefig('../results/figures/n_components_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Discussion

### Key Findings:
1. **PCA Effectiveness**: PCA captures key variance in MRI data and reduces dimensionality
2. **Computational Efficiency**: Clustering in reduced space is computationally efficient
3. **Segmentation Quality**: Compare PCA-based vs original space segmentation
4. **Eigenvector Interpretation**: Principal components correspond to meaningful image features

### Relationship between Eigenvalues/Eigenvectors and Image Features:
- **Largest eigenvalues** capture the most variance in tissue contrast
- **Eigenvectors** represent directions of maximum variance
- **First few PCs** often capture intensity differences between tissue types
- **Higher PCs** may capture noise or fine details

## 11. Save Results

In [ ]:
# Save segmentation results
np.save('../results/segmentations/segmentation_pca.npy', results['segmentation'])
np.save('../results/segmentations/segmentation_original.npy', seg_original)

# Save statistics
stats_df.to_csv('../results/tissue_statistics.csv')

print("Results saved to ../results/")